# 🧠 Convert .mat to PNG (MRI, Mask, Border)

This notebook reads `.mat` files from the Figshare brain tumor dataset and generates high-quality `.png` images:

- Grayscale MRI
- MRI + Tumor Border (in red)
- MRI + Tumor Mask Overlay (semi-transparent red)
- Binary Tumor Mask (0 or 255)

📁 Outputs are saved to:
- `mri/`
- `border/`
- `mask/`
- `binary_mask/`

📌 Run locally – update file paths as needed.


In [3]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt

# === Local path to your raw .mat dataset ===
mat_folder = "/Users/aarogyagyawali/Downloads/Figshare datasets/BrainTumorPositive_DS_Matlab"

# === Local output folders to save split images ===
base_output = "/Users/aarogyagyawali/Downloads/BrainScanAI_final_output"
mri_folder = os.path.join(base_output, "mri")
border_folder = os.path.join(base_output, "border")
mask_folder = os.path.join(base_output, "mask")

# === Create output folders if not already present ===
os.makedirs(mri_folder, exist_ok=True)
os.makedirs(border_folder, exist_ok=True)
os.makedirs(mask_folder, exist_ok=True)


# List all .mat files
mat_files = sorted([f for f in os.listdir(mat_folder) if f.endswith('.mat')])

# Loop through all files
for i, file_name in enumerate(mat_files):
    file_path = os.path.join(mat_folder, file_name)
    base_name = os.path.splitext(file_name)[0]

    try:
        with h5py.File(file_path, 'r') as f:
            cjdata = f['cjdata']
            image = np.array(cjdata['image']).T
            tumor_mask = np.array(cjdata['tumorMask']).T
            border_data = np.array(cjdata['tumorBorder'])

            # Flatten and reshape tumor border safely
            flat_border = border_data.flatten()
            if len(flat_border) % 2 != 0:
                print(f"⚠️ Skipped malformed border in: {file_name}")
                continue
            tumor_border = flat_border.reshape(-1, 2)

        # --- Save Original MRI ---
        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        ax.axis('off')
        fig.savefig(os.path.join(mri_folder, base_name + ".png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        # --- Save Tumor Border Overlay ---
        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        ax.plot(tumor_border[:, 0], tumor_border[:, 1], color='red', linewidth=2)
        ax.axis('off')
        fig.savefig(os.path.join(border_folder, base_name + ".png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        # --- Save Mask Overlay ---
        fig, ax = plt.subplots()
        ax.imshow(image, cmap='gray')
        ax.imshow(tumor_mask, cmap='Reds', alpha=0.5)
        ax.axis('off')
        fig.savefig(os.path.join(mask_folder, base_name + ".png"), bbox_inches='tight', pad_inches=0)
        plt.close(fig)

        plt.close('all')

        if i % 100 == 0:
            print(f"✅ Processed {i}/{len(mat_files)}: {file_name}")

    except Exception as e:
        print(f"❌ Error processing {file_name}: {e}")

✅ Processed 0/3064: 1.mat
✅ Processed 100/3064: 1089.mat
✅ Processed 200/3064: 1179.mat
✅ Processed 300/3064: 1269.mat
✅ Processed 400/3064: 1359.mat
✅ Processed 500/3064: 1449.mat
✅ Processed 600/3064: 1539.mat
✅ Processed 700/3064: 1629.mat
✅ Processed 800/3064: 1719.mat
✅ Processed 900/3064: 1809.mat
✅ Processed 1000/3064: 19.mat
✅ Processed 1100/3064: 199.mat
✅ Processed 1200/3064: 2079.mat
✅ Processed 1300/3064: 2169.mat
✅ Processed 1400/3064: 2259.mat
✅ Processed 1500/3064: 2349.mat
✅ Processed 1600/3064: 2439.mat
✅ Processed 1700/3064: 2529.mat
✅ Processed 1800/3064: 2619.mat
✅ Processed 1900/3064: 2709.mat
✅ Processed 2000/3064: 28.mat
✅ Processed 2100/3064: 289.mat
✅ Processed 2200/3064: 298.mat
✅ Processed 2300/3064: 310.mat
✅ Processed 2400/3064: 400.mat
✅ Processed 2500/3064: 491.mat
✅ Processed 2600/3064: 581.mat
✅ Processed 2700/3064: 671.mat
✅ Processed 2800/3064: 761.mat
✅ Processed 2900/3064: 851.mat
✅ Processed 3000/3064: 941.mat


In [5]:
binary_mask_folder = "/Users/aarogyagyawali/Downloads/BrainScanAI_final_output/binary_mask"

# === Create binary mask output folder ===
os.makedirs(binary_mask_folder, exist_ok=True)

# === List all .mat files ===
mat_files = sorted([f for f in os.listdir(mat_folder) if f.endswith(".mat")])

# === Generate and save binary masks ===
for i, file_name in enumerate(mat_files):
    file_path = os.path.join(mat_folder, file_name)
    base_name = os.path.splitext(file_name)[0]

    try:
        with h5py.File(file_path, 'r') as f:
            cjdata = f["cjdata"]
            tumor_mask = np.array(cjdata["tumorMask"]).T  # Transpose to match original image orientation

        # Convert mask to binary (0 and 255)
        binary_mask = (tumor_mask > 0).astype(np.uint8) * 255
        Image.fromarray(binary_mask).save(os.path.join(binary_mask_folder, base_name + ".png"))

        if i % 100 == 0:
            print(f"✅ Saved binary mask: {file_name}")

    except Exception as e:
        print(f"❌ Error in {file_name}: {e}")

print("🏁 Binary mask generation complete.")

✅ Saved binary mask: 1.mat
✅ Saved binary mask: 1089.mat
✅ Saved binary mask: 1179.mat
✅ Saved binary mask: 1269.mat
✅ Saved binary mask: 1359.mat
✅ Saved binary mask: 1449.mat
✅ Saved binary mask: 1539.mat
✅ Saved binary mask: 1629.mat
✅ Saved binary mask: 1719.mat
✅ Saved binary mask: 1809.mat
✅ Saved binary mask: 19.mat
✅ Saved binary mask: 199.mat
✅ Saved binary mask: 2079.mat
✅ Saved binary mask: 2169.mat
✅ Saved binary mask: 2259.mat
✅ Saved binary mask: 2349.mat
✅ Saved binary mask: 2439.mat
✅ Saved binary mask: 2529.mat
✅ Saved binary mask: 2619.mat
✅ Saved binary mask: 2709.mat
✅ Saved binary mask: 28.mat
✅ Saved binary mask: 289.mat
✅ Saved binary mask: 298.mat
✅ Saved binary mask: 310.mat
✅ Saved binary mask: 400.mat
✅ Saved binary mask: 491.mat
✅ Saved binary mask: 581.mat
✅ Saved binary mask: 671.mat
✅ Saved binary mask: 761.mat
✅ Saved binary mask: 851.mat
✅ Saved binary mask: 941.mat
🏁 Binary mask generation complete.
